In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [2]:
IMG_SIZE = 64
TRAIN_PATH = r'C:\Users\me\Downloads\archive\train'
TEST_PATH = r'C:\Users\me\Downloads\archive\test'

In [10]:
def load_images_from_folder_flat(folder_path, max_images_per_class=500):
    X, y = [], []
    dog_count = 0
    cat_count = 0

    for filename in os.listdir(folder_path):
        if not filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue

        img_path = os.path.join(folder_path, filename)

        if 'dog' in filename.lower() and dog_count < max_images_per_class:
            label = 1
            dog_count += 1
        elif 'cat' in filename.lower() and cat_count < max_images_per_class:
            label = 0
            cat_count += 1
        else:
            continue  # Skip if over limit or not a dog/cat

        try:
            img = cv2.imread(img_path)
            if img is None:
                continue
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            X.append(img.flatten())
            y.append(label)
        except Exception as e:
            print(f"❌ Error reading {img_path}: {e}")
            continue

    return np.array(X), np.array(y)


In [11]:
X_train, y_train = load_images_from_folder_flat(TRAIN_PATH, max_images_per_class=500)
X_test, y_test = load_images_from_folder_flat(TEST_PATH, max_images_per_class=200)

print("✅ Training images:", len(X_train))
print("✅ Test images:", len(X_test))

✅ Training images: 1000
✅ Test images: 400


In [12]:
import collections

print("Class distribution in training data:", collections.Counter(y_train))
print("Class distribution in test data:", collections.Counter(y_test))


Class distribution in training data: Counter({np.int64(0): 500, np.int64(1): 500})
Class distribution in test data: Counter({np.int64(0): 200, np.int64(1): 200})


In [22]:
model = SVC(kernel='linear')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Classification Report:', classification_report(y_test, y_pred, target_names=['Cat', 'Dog']))

Accuracy: 0.5325
Classification Report:               precision    recall  f1-score   support

         Cat       0.53      0.58      0.56       200
         Dog       0.54      0.48      0.51       200

    accuracy                           0.53       400
   macro avg       0.53      0.53      0.53       400
weighted avg       0.53      0.53      0.53       400



In [21]:
def predict_single_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)).flatten().reshape(1, -1)
    pred = model.predict(img)
    return 'Dog 🐶' if pred[0] == 1 else 'Cat 🐱'

# Example:
predict_single_image(r'C:\Users\me\Downloads\archive\test\cat.180.jpg')


'Cat 🐱'